In [1]:
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn

In [2]:
mlflow.set_tracking_uri('./mlruns')
mlflow.set_experiment('singlts')
experiment = mlflow.get_experiment_by_name('singlets')
client = MlflowClient()

run = client.create_run(experiment.experiment_id)

with mlflow.start_run(run_id=run.info.run_id) as run:

    model_lightGBM.fit(series=train_dataset_ts_prepared)
    #signature = infer_signature(test_dataset_ts_prepared, model_lightGBM.predict(n=forecast_horizons, series=train_dataset_ts_prepared))
    
    mlflow.lightgbm.log_model(model_lightGBM, 'model')

ValueError: could not convert string to float: '--ip=127.0.0.1'